Automatic alignment of time using displayed time of iOS devices

In [ ]:
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
import json
import datetime
import os
%matplotlib notebook
from pymongo import MongoClient, UpdateMany, UpdateOne, InsertOne
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

In [ ]:
url='mongodb://192.168.0.94:27017/'
client = MongoClient(url)
tbl = client['global']['keynotes']
tbl2 = client['global']['timekeys']

In [ ]:
video_hd_folder = '../../../../data/video_hd'
ocr_folder = '../../../../data/ocr'
timealignment_folder = '../../../../data/timealignment'

In [ ]:
if not os.path.exists(timealignment_folder):
    os.makedirs(timealignment_folder)

In [ ]:
def get_stock_time(selectedframe, showntime):
    global currentstate
    print('selected frame',selectedframe)
    
    video_time = selectedframe/currentstate['fps']

    
    first_time = (int(showntime.split(':')[0])+3)*3600+int(showntime.split(':')[1])*60
    if len(showntime.split(':')) == 3:
        first_time += int(showntime.split(':')[2])
    stock_time = (first_time - video_time)*1000
    print(video_time, stock_time)
    return video_time, stock_time

In [ ]:
ls = list(tbl.find({'ocr_downloaded':1}))

In [ ]:
def load_video(state):
    ii = ls[state['videonr']]
    state['id'] = ii['id']
    print('Analysing video',state['id'])

    cap = cv2.VideoCapture(video_hd_folder+'/'+state['id']+'.mp4')
    fps = cap.get(cv2.CAP_PROP_FPS)
    print ("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))

    # Import OCR file of video
    j = ""
    with open(ocr_folder+'/'+ii['id']+'.json', 'r') as handle:
        j = json.load(handle)
        
    state['j'] = j
    
    tia = ['10:','11:','12:']

    results = []
    # Search for timestamps
    for ai in range(len(j['response']['annotationResults'])):
        a = j['response']['annotationResults'][ai]
        for bi in range(len(a['textAnnotations'])):
            b = a['textAnnotations'][bi]
            s = float(b['segments'][0]['segment']['startTimeOffset'].replace('s',''))
            r = datetime.datetime.combine(datetime.date.today(), datetime.time(hour=10))+datetime.timedelta(seconds=s)
            if any([q in b['text'] for q in tia]):
                results.append({'ai':ai,'bi':bi,'time':r.strftime("%H:%M:%S")})
    state['results'] = results
    state['ri'] = -1
    state['fps'] = fps
    state['cap'] = cap

In [ ]:
def prev_elem():
    global currentstate
    currentstate['ri']-=2
    if currentstate['ri']<=-2:
        print('No more previous states - Loading first state')
        currentstate['ri']=-1
    next_elem()

In [ ]:
def next_elem(*argv):
    global currentstate
    if 'j' not in currentstate:
        load_video(currentstate)
    currentstate['ri']+=1
    if currentstate['ri']>=len(currentstate['results']):
        currentstate['videonr']+=1
        del currentstate['j']
        return next_elem()
    
    j = currentstate['j']
    res = currentstate['results'][currentstate['ri']]
    iid = res['bi']
    segid = 0
    ocr = j['response']['annotationResults'][res['ai']]
    currentstate['ocr_iid'] = ocr['textAnnotations'][iid]
    seg = currentstate['ocr_iid']['segments'][segid]
    frame = seg['frames'][segid]
    currentstate['frame'] = frame
    fps = currentstate['fps']
    

    s = float(frame['timeOffset'].replace('s',''))
    currentstate['framenum'] = s*fps


In [ ]:
def draw_widgets():
    global currentstate
    l = widgets.Label(value='loading')
    a = widgets.IntSlider(value=0,min=-200,max=200)
    b = widgets.IntSlider(value=0,min=-50,max=50)
    c = widgets.IntSlider(value=0,min=-50,max=50)
    d = widgets.Button(
        description='Prev',
        disabled=False,
        button_style='', 
        tooltip='Click me',
    )
    e = widgets.Button(
        description='Next',
        disabled=False,
        button_style='', 
        tooltip='Click me',
    )
    f = widgets.Button(
        description='SetTimeChange',
        disabled=False,
        button_style='', 
        tooltip='Click me',
    )
    g = widgets.Button(
        description='Next video',
        disabled=False,
        button_style='', 
        tooltip='Click me',
    )
    t = widgets.Text(
        value=currentstate['ocr_iid']['text'],
        placeholder='HH:MM',
        description='Please enter the starting (new) minute here (HH:MM[:SS]):',
        disabled=False
    )
    def refresh():
        a.value = 0
        b.value = 0
        c.value = 0
        t.value = currentstate['ocr_iid']['text']
        f.description = 'Save '+str(currentstate['ri'])+'/'+str(len(currentstate['results']))
        l.value = currentstate['id']+' @ '+currentstate['results'][currentstate['ri']]['time']
        
        
    def nxt(x):
        next_elem()
        refresh()
        
    def prv(x):
        prev_elem()
        refresh()
    def nv(x):
        currentstate['ri'] = len(currentstate['results'])-1
        nxt(x)
        
    def save(x):
        ret = get_stock_time(currentstate['framenum']+a.value, t.value)
        qry = {'id':currentstate['id'], 'timekey':ret, 'source2':{'ri':currentstate['ri'],
                                                                 'res':currentstate['results'][currentstate['ri']]}}
        tbl2.insert_one(qry)
        print('Saved in db', qry)
        next_elem()
        refresh()
        
    d.on_click(prv)
    e.on_click(nxt)
    f.on_click(save)
    g.on_click(nv)
    
    ui = widgets.VBox([l,widgets.HBox([d,e,f,g]),widgets.HBox([t]),widgets.HBox([a, b, c])])
    
    out = widgets.interactive_output(draw_state, {'a': a, 'b': b, 'c': c, 'd':t})

    display(ui, out)
    refresh()
    

In [ ]:
def draw_state(a,b,c,d):
    global currentstate
    deltaframe = a
    deltapos = (b*10,c*10)
    
    framenum = currentstate['framenum']
    framenum += deltaframe
    cap = currentstate['cap']
    cap.set(cv2.CAP_PROP_POS_FRAMES, framenum)

    
    frame = currentstate['frame']
    
    _,img = cap.read()
    plt.rcParams["figure.figsize"] = (10,5)
    
    for a in frame['rotatedBoundingBox']['vertices']:
        if 'y' not in a:
            a['y']=0
    box = [(int((a['x'])*img.shape[1])+deltapos[0],int((a['y'])*img.shape[0])+deltapos[1]) for a in frame['rotatedBoundingBox']['vertices']]
    
    a = 15
    simg = img[max(box[0][1]-a,0):min(box[3][1]+a,img.shape[0]),max(box[0][0]-a,0):min(box[1][0]+a,img.shape[1])]
    
    caption = ''

    if caption == '':
        plt.imshow(img)
        plt.show()
            
    plt.figure()
    try:
        plt.imshow(simg)
    except:
        print('error displaying frame, maybe box is not valid; display whole image instead',box)
        plt.imshow(img)
    
    fn = cap.get(cv2.CAP_PROP_POS_FRAMES)
    plt.title(caption+' - '+str(fn))
    plt.show()


In [ ]:
currentstate = {
    'videonr':0,
}
next_elem()

In [ ]:
draw_widgets()

In [ ]:
# Set finished flag for ALL SELECTED !!
tbl.update_many({'selected':1},{'$set':{'time_aligned':1}})

In [ ]:
def display_iid(ocr_iid, segid, framenum=None, caption=''):
    seg = ocr_iid['segments'][segid]
    frame = seg['frames'][segid]
    if framenum is None:
        s = float(frame['timeOffset'].replace('s',''))
        framenum = s*fps
    cap.set(cv2.CAP_PROP_POS_FRAMES, framenum)

    _,img = cap.read()
    plt.rcParams["figure.figsize"] = (10,5)
    
    for a in frame['rotatedBoundingBox']['vertices']:
        if 'y' not in a:
            a['y']=0
    box = [(int((a['x'])*img.shape[1]),int((a['y'])*img.shape[0])) for a in frame['rotatedBoundingBox']['vertices']]
    
    a = 150
    simg = img[max(box[0][1]-a,0):min(box[3][1]+a,img.shape[0]),max(box[0][0]-a,0):min(box[1][0]+a,img.shape[1])]
    
    if caption == '':
        plt.imshow(img)
        plt.show()
            
    plt.figure()
    try:
        plt.imshow(simg)
    except:
        print('error displaying frame, maybe box is not valid; display whole image instead',box)
        plt.imshow(img)
    
    fn = cap.get(cv2.CAP_PROP_POS_FRAMES)
    plt.title(caption+' - '+str(fn))
    plt.show()
    
    if caption == '':
        for i in range(-3,0):
            display_iid(ocr_iid, segid,  framenum=fn-1.001*i, caption=str(i))
        for i in range(1,10):
            display_iid(ocr_iid, segid,  framenum=fn-1.001*i, caption=str(i))
            
        
    return fn, ocr_iid['text']

In [ ]:
def to_orig_time(timekey):
    k = round(timekey[1]/1000+timekey[0],4)
    hh = int(k/3600-3)
    mm = int((k%3600)/60)
    ss = k%3600%60
    return str(hh)+':'+str(mm)+':'+str(ss)

In [ ]:
# Print statistics
ls = list(tbl.find({'time_aligned':1}))
for i in ls:
    ls2 = list(tbl2.find({'id':i['id']}))
    print('Analysis for', i['id'])
    sources = {}
    results = []
    xresult = []
    ls2 = sorted(ls2, key=lambda tup: tup['timekey'][0])
    
    for ii in ls2:
        #assert ii['source'] not in sources # Timekey is twice in DB
        #sources[ii['source']] = 1
        xresult.append(ii['timekey'][0]/60)
        results.append(ii['timekey'][1])
    plt.plot(xresult,results)
    plt.show()
    print('nr of points,mean,std, (should be around 33), times in ms:',len(results),np.mean(results),np.std(results))
    
    mean = np.mean(results)
    # do deeper anaylsis
    if True or input('Show deeper analysis? [yn]')=='y':
        cap = cv2.VideoCapture(video_hd_folder+'/'+i['id']+'.mp4')
        fps = cap.get(cv2.CAP_PROP_FPS)

        j = ""
        with open(ocr_folder+'/'+i['id']+'.json', 'r') as handle:
            j = json.load(handle)
        for ii in ls2:
            nr = ''
            if 'source' in ii:
                ocr = j['response']['annotationResults'][0]
                ocr_iid =  ocr['textAnnotations'][ii['source']]
            else:
                ocr = j['response']['annotationResults'][ii['source2']['res']['ai']]
                iid = ii['source2']['res']['bi']
                ocr_iid =  ocr['textAnnotations'][iid]
                nr = '('+str(ii['source2']['ri'])+') '
                
            display_iid(ocr_iid, 0,  
                           framenum=ii['timekey'][0]*fps-1.001, caption=(nr+'before - dist from mean:'+str(ii['timekey'][1]-mean)))
            display_iid(ocr_iid, 0, 
                           framenum=ii['timekey'][0]*fps, caption=nr+'after - should show:'+to_orig_time(ii['timekey']))

In [ ]:
# Reset flags with no timekey
tk = {}
ls = list(tbl.find({'time_aligned':1}))
for i in ls:
    tk[i['id']] = 1
ls2 = list(tbl2.find({}))
for i in ls2:
    if i['id'] in tk:
        k = 1
        del tk[i['id']]
for i in tk:
    tbl.update_one({'id':i},{'$unset':{'time_aligned':''}})

In [ ]:
# Save results to text file for import
ls = list(tbl.find({'time_aligned':1}))
for i in ls:
    ls2 = list(tbl2.find({'id':i['id']}))
    for ii in ls2:
        del ii['_id']
    with open(timealignment_folder+'/'+i['id']+'.json','w') as f:
        json.dump(ls2,f)    